In [2]:
#loading the packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import spacy
from collections import Counter
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import time
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer 

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from textblob import TextBlob
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import langdetect
from langdetect import detect
import detectlanguage
from google_trans_new import google_translator  
import langid

In [353]:
#loading development datasets and concatenating thhem
dev_de = pd.read_json("dataset_de_dev.json", lines=True)
dev_es = pd.read_json("dataset_es_dev.json", lines=True)
dev_en = pd.read_json("dataset_en_dev.json", lines=True)
dev_fr = pd.read_json("dataset_fr_dev.json", lines=True)
dev= pd.concat([dev_fr, dev_es, dev_de, dev_en], axis=0)
dev.reset_index(drop=True, inplace=True)
dev

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,fr_0112905,product_fr_0620743,reviewer_fr_0310864,1,Colis bien reçus avec le boîtier du jeu ouvert...,Commandé ailleurs si vous voulez des colis com...,fr,video_games
1,fr_0345365,product_fr_0942745,reviewer_fr_0244013,1,"Suite à une vérification, je vous informe que ...",COMMANDE NON RECUE,fr,office_product
2,fr_0474170,product_fr_0183830,reviewer_fr_0816879,1,"C'est un produit de piètre qualité sur quatre,...",Plastique sans aucune résistance !,fr,automotive
3,fr_0084156,product_fr_0195761,reviewer_fr_0933238,1,N'ont pas résistés. Piètre qualité,séparateur d'orteils,fr,sports
4,fr_0893613,product_fr_0077995,reviewer_fr_0492122,1,"Filament de qualité mais pas bien embobiné, ré...","ça coince, ça coince... que des problemes",fr,industrial_supplies
...,...,...,...,...,...,...,...,...
19995,en_0447642,product_en_0068982,reviewer_en_0061521,5,This ribbon is so adorable! Goes perfect with ...,So Cute!,en,home
19996,en_0950370,product_en_0563046,reviewer_en_0871798,5,I am in love with this kettle.,Perfect,en,kitchen
19997,en_0203466,product_en_0848682,reviewer_en_0474236,5,My Doberman Loves Having His Nails Trimmed and...,My Doberman Loves Them,en,pet_products
19998,en_0010627,product_en_0536493,reviewer_en_0546192,5,"I love my Fire.. I do everything on it, read, ...",Five Stars,en,other


In [355]:
#loading data pre-processed with unique linguistic characteristics
train=pd.read_excel('train_df_clean.xlsx')

In [270]:
#training data with 
train_nostop=pd.read_excel('no_stopw.xlsx')


In [271]:
#cleaning the data set
train_nostop= train_nostop[train_nostop['review_body'].notnull()]
train_nostop.reset_index(drop=True, inplace=True)

In [262]:
#training data with all stop words
train_sw=pd.read_excel('train_sw.xlsx')

In [280]:
#traiing set with stemmed data
train_lemma=pd.read_excel('train_lemma.xlsx')

In [354]:
#test set
test=pd.read_excel('test.xlsx')

In [295]:
#Europarl long
formal_large=pd.read_excel('formal_large.xlsx')

In [211]:
formal_short_nostop=pd.read_excel('formal_nostop.xlsx')

In [217]:
functional=pd.read_excel('functional_words.xlsx')
functional.head(2)

,Unnamed: 0,text,lang
0,0,want imagine wearable,en
1,1,Berkeley Bionics call,en


In [219]:
functional=functional[functional['text'].str.split().str.len().ge(2)]
functional= functional[functional['text'].notnull()]

functional.reset_index(drop=True, inplace=True)

In [222]:
functional.shape

(106182, 3)

In [223]:
functional['lang'].value_counts()

fr    28186
en    28083
de    25346
es    24567
Name: lang, dtype: int64

In [225]:
ted_talk_full=pd.read_excel('tedtalk_full_large.xlsx')
ted_talk_full=ted_talk_full[ted_talk_full['text'].str.split().str.len().ge(4)]
ted_talk_full= ted_talk_full[ted_talk_full['text'].notnull()]
ted_talk_full.reset_index(drop=True, inplace=True)
ted_talk_full.head(2)

,Unnamed: 0,text,lang
0,0,I want you now to imagine a wearable robot tha...,en
1,1,We at Berkeley Bionics call these robots exosk...,en


In [226]:
ted_talk_full.shape

(124157, 3)

In [227]:
ted_talk_full['lang'].value_counts()

en    32156
fr    31966
es    30680
de    29355
Name: lang, dtype: int64

Testing the performance of MNB for monolingual pre-processed data

In [348]:
#training set with monolingual pre-processing
train_mono=pd.read_excel('train_mono.xlsx')
train_mono= train_mono[train_mono['review_body'].notnull()]
train_mono.reset_index(drop=True, inplace=True)

In [352]:
#training and testing MNB over the data pre-processed as monolingual data
train_X = train_mono['review_body']   
train_y = train_mono['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None, max_features=10000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
lr = LogisticRegression()
lr.fit(X_train_tf, train_y)
y_pred = lr.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=lr.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

import pickle
filename_lr = 'model_lr_10k.sav'
pickle.dump(lr, open(filename_lr, 'wb'))

# load the model from disk
loaded_model_train_lr = pickle.load(open(filename_lr, 'rb'))

x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_lr.score(x_formal, formal_large.lang)
print(result)

x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_lr.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=lr.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=lr.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

accuracy test set:   0.84284999999999998810
[[4818  103   72    7]
 [ 432 4464   69   35]
 [1423   27 3269  281]
 [ 320   27  347 4306]]
------------------------------
accuracy validation set:  0.84745000000000003659
[[4814  116   65    5]
 [ 425 4481   63   31]
 [1334   22 3336  308]
 [ 290   27  365 4318]]
0.4894280498935199
0.4819906843804584
------------------------------
accuracy validation set:  0.48127773705872400889
[[19995  4564  3323  1473]
 [12760 16979   960  1457]
 [19231   391  8347  2711]
 [12298   652  4583 14433]]
------------------------------
accuracy validation set:  0.51847770808611626236
[[23870   799   514   163]
 [ 8570 17328  1381   804]
 [21519   341  2346   361]
 [15500   596   581 11509]]


In [342]:
#loading the model
import pickle
filename_10k = 'model_train_mono_10k.sav'
pickle.dump(naive_bayes_classifier, open(filename_10k, 'wb'))

In [343]:
# load the model from disk
loaded_model_mono_10k = pickle.load(open(filename_10k, 'rb'))

Performance of MNB with 10,000 features and trained over the data pre-processed accounting for linguistic nuances

In [233]:
tvec_up_more = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=1000000, ngram_range=(1, 3))
#pipeline for 1,000,000 features
clf = MultinomialNB()
bayes_pipeline_up_more = Pipeline([
    ('vectorizer', tvec_up_more),
    ('classifier', clf)
])

In [89]:
#Stratified k-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_up_more.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99885109 0.99992499 1.         0.99987493]
recall:    [0.99989999 0.99987499 0.99942494 0.99944994]
f1 score:  [0.99937527 0.99989999 0.99971239 0.99966239]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99850165 0.99960007 0.99997498 0.99994996]
recall:    [0.99969997 0.99987499 0.99917492 0.99927493]
f1 score:  [0.99910045 0.99973751 0.99957479 0.99961233]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99895087 0.99984999 0.99997498 0.99989996]
recall:    [0.99987499 0.99989999 0.99932493 0.99957496]
f1 score:  [0.99941271 0.99987499 0.99964985 0.99973743]
--------------------------------------------------------
              de          en          es       fr
precision: [0.9992005  0.99989999 0.99997499 0.99989996]
recall:    [0.99992499 0.99987499 0.99949995 0.99967496]
f1

In [356]:
#training and testing the model on test and dev sets
train_X = train['new']   
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=10000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99970000000000003304
[[5000    0    0    0]
 [   1 4999    0    0]
 [   1    0 4999    0]
 [   4    0    0 4996]]
------------------------------
accuracy validation set:  0.99980000000000002203
[[4998    2    0    0]
 [   0 5000    0    0]
 [   1    0 4999    0]
 [   1    0    0 4999]]


In [235]:
#loading the model
import pickle
filename_10k = 'model_train_tfidf_10k.sav'
pickle.dump(naive_bayes_classifier, open(filename_10k, 'wb'))

In [357]:
#performance on long and short EuroParl, performance on long and short TED Talks sample
import pickle
filename_10k_new = 'model_train_tfidf_10k_new.sav'
pickle.dump(naive_bayes_classifier, open(filename_10k_new, 'wb'))

# load the model from disk
loaded_model_train_10k_new = pickle.load(open(filename_10k_new, 'rb'))

x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_10k_new.score(x_formal, formal_large.lang)
print(result)

x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_10k_new.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))



0.9967168644153737
0.8060128089768698
------------------------------
accuracy validation set:  0.97630419549441438409
[[29222   103    10    20]
 [   86 32028    10    32]
 [ 1900   106 28587    87]
 [  480    67    41 31378]]
------------------------------
accuracy validation set:  0.92291537172025395197
[[25067   177    34    68]
 [ 1167 26546   112   258]
 [ 4194   104 20183    86]
 [ 1762   136    87 26201]]


In [314]:
#testing the baseline langid.py
import langid
from sklearn import metrics
test_X = test['review_body']
langid.set_languages(['de','fr','es','en'])
langs=test['review_body'].apply(lambda x: langid.classify(x))
first_tuple_elements = []
for lang in langs:
    first_tuple_elements.append(lang[0])

In [320]:
score2 = metrics.accuracy_score(test.language,first_tuple_elements)

In [321]:
#accuracy for the test set
score2

0.9972

In [324]:
#langid.py performance on long Europarl 
import langid
from sklearn import metrics
#test_X = test['review_body']
langid.set_languages(['de','fr','es','en'])
langs=formal_large['txt'].apply(lambda x: langid.classify(x))
first_tuple_elements = []
for lang in langs:
    first_tuple_elements.append(lang[0])
score3=metrics.accuracy_score(formal_large.lang,first_tuple_elements)

In [325]:
#accuracy for long Europarl
print("accuracy validation set:  %0.20f" % score3)

accuracy validation set:  0.99816195112057604089


In [326]:
#testing for short Europarl
import langid
from sklearn import metrics
langid.set_languages(['de','fr','es','en'])
langs=formal_short_nostop['txt'].apply(lambda x: langid.classify(x))
first_tuple_elements = []
for lang in langs:
    first_tuple_elements.append(lang[0])
score4=metrics.accuracy_score(formal_short_nostop.lang,first_tuple_elements)

In [327]:
#accuracy for short Europarl
print("accuracy validation set:  %0.20f" % score4)

accuracy validation set:  0.93252792039379661659


In [328]:
#testing langid.py on long TED Talks
import langid
from sklearn import metrics
langid.set_languages(['de','fr','es','en'])
langs=ted_talk_full['text'].apply(lambda x: langid.classify(x))
first_tuple_elements = []
for lang in langs:
    first_tuple_elements.append(lang[0])
score5=metrics.accuracy_score(ted_talk_full.lang,first_tuple_elements)

In [329]:
#accuracy for long TED Talks
print("accuracy validation set:  %0.20f" % score5)

accuracy validation set:  0.99015762300957654851


In [330]:
#testing on short TED Talks sample
import langid
from sklearn import metrics
langid.set_languages(['de','fr','es','en'])
langs=functional['text'].apply(lambda x: langid.classify(x))
first_tuple_elements = []
for lang in langs:
    first_tuple_elements.append(lang[0])
score6=metrics.accuracy_score(functional.lang,first_tuple_elements)

In [331]:
#accuracy for the short TED Talks sample
print("accuracy validation set:  %0.20f" % score6)

accuracy validation set:  0.94658228324951498855


MNB with 100,000 features

In [59]:
tvec_up_more = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=100000, ngram_range=(1, 3))
#pipeline for 100,000 features
clf = MultinomialNB()
bayes_pipeline_up_more = Pipeline([
    ('vectorizer', tvec_up_more),
    ('classifier', clf)
])

In [60]:
#stratified cross validation for 100,000
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_up_more.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99987499 0.99995    1.         0.99994999]
recall:    [0.99992499 0.999975   0.99992499 0.99994999]
f1 score:  [0.99989999 0.9999625  0.99996249 0.99994999]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99977497 0.99957513 0.99994999 0.99997499]
recall:    [0.99972497 0.999975   0.99984998 0.99972497]
f1 score:  [0.99974997 0.99977502 0.99989998 0.99984997]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99972502 0.99985    1.         0.99997499]
recall:    [0.99989999 0.99994999 0.99982498 0.99987499]
f1 score:  [0.9998125  0.99989999 0.99991248 0.99992499]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99987499 0.99992499 0.999975   0.99994999]
recall:    [0.99992499 0.99994999 0.99994999 0.99989999]
f1

In [244]:
#training and testing the model for 100,000
train_X = train['new']   
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=100000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99995000000000000551
[[5000    0    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   1    0    0 4999]]
------------------------------
accuracy validation set:  0.99995000000000000551
[[4999    1    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   0    0    0 5000]]


In [245]:
#saving into a pickle
import pickle
filename_100k = 'model_train_tfidf_100k.sav'
pickle.dump(naive_bayes_classifier, open(filename_100k, 'wb'))

In [246]:
# load the model from disk
loaded_model_train_100k = pickle.load(open(filename_100k, 'rb'))


In [247]:
#accuracy for long Europarl
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_100k.score(x_formal, formal_large.lang)
print(result)

0.9991887232532197


In [248]:
#accuracy for short Europarl
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_100k.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9333218652410946


In [249]:
#accuracy for TED Talks long texts
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.99050395869745566646
[[29244    90     9    12]
 [   30 32087     5    34]
 [  581   121 29925    53]
 [  143    75    26 31722]]


In [250]:
#accuracies for short TED Talks
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.97508052212239360035
[[25027   214    34    71]
 [  274 27596    63   150]
 [ 1016   131 23323    97]
 [  416   152    28 27590]]


Testing MNB with 1,000,000 features

In [19]:
#stratified k-fold validation  
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_up_more.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99992499 0.99995    1.         0.999975  ]
recall:    [0.99994999 0.999975   0.999975   0.99994999]
f1 score:  [0.99993749 0.9999625  0.9999875  0.9999625 ]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99987497 0.99967506 1.         0.999975  ]
recall:    [0.99972497 0.999975   0.99992499 0.99989999]
f1 score:  [0.99979996 0.99982501 0.99996249 0.99993749]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99994999 0.99985    1.         0.99994999]
recall:    [0.99989999 0.999975   0.99992499 0.99994999]
f1 score:  [0.99992499 0.9999125  0.99996249 0.99994999]
--------------------------------------------------------
              de          en          es       fr
precision: [0.999975 0.999975 0.999975 0.999975]
recall:    [0.999975   0.99994999 1.         0.999975  ]
f1 score: 

In [254]:
#training with the max features set to 1,000,000 and testing on testand dev sets
train_X = train['new']  
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=1000000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99995000000000000551
[[5000    0    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   1    0    0 4999]]
------------------------------
accuracy validation set:  0.99995000000000000551
[[4999    1    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   0    0    0 5000]]


In [255]:
#saving model to a pickle
import pickle
filename = 'model_train_tfidf.sav'
pickle.dump(naive_bayes_classifier, open(filename, 'wb'))

In [256]:
# load the model from disk
loaded_model_train = pickle.load(open(filename, 'rb'))


In [257]:
#accuracy for long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train.score(x_formal, formal_large.lang)
print(result)

0.9995309806307676


In [258]:
#accuracy for short Europarl sample
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9755861959455883


In [259]:
#accuracy for long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.99550569037589509147
[[29249    86    10    10]
 [   16 32112     3    25]
 [  130   127 30389    34]
 [   27    78    12 31849]]


In [33]:
#accuracy for short TED Talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.98978169557928841193
[[25020   231    29    66]
 [   61 27892    44    86]
 [  148   143 24217    59]
 [   54   144    20 27968]]


Model with data with all stop words

In [36]:
#stratified k-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train_sw.review_body
Y=train_sw.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_up_more.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99992499 0.99995    0.999975   0.999975  ]
recall:    [0.99992499 0.999975   0.999975   0.99994999]
f1 score:  [0.99992499 0.9999625  0.999975   0.9999625 ]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99997499 0.99967506 0.999975   0.999975  ]
recall:    [0.99972497 0.999975   0.999975   0.99992499]
f1 score:  [0.99984997 0.99982501 0.999975   0.99994999]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99994999 0.9999     1.         0.999975  ]
recall:    [0.99989999 0.999975   0.999975   0.999975  ]
f1 score:  [0.99992499 0.9999375  0.9999875  0.999975  ]
--------------------------------------------------------
              de          en          es       fr
precision: [0.999975 1.       0.999975 0.999975]
recall:    [1.       0.999975 0.999975 0.999975]
f1 score:  [0.9999

In [94]:
#training MNB over the data with all stop words preserved
train_X = train_sw['review_body']   
train_y = train_sw['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b', max_features=1000000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99990000000000001101
[[5000    0    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   1    0    1 4998]]
------------------------------
accuracy validation set:  0.99995000000000000551
[[4999    1    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   0    0    0 5000]]


In [263]:
#saving the model to a pickle
import pickle
filename1 = 'model__sw_train_tfidf.sav'
pickle.dump(naive_bayes_classifier, open(filename1, 'wb'))

In [264]:
# load the model from disk
loaded_model_train_sw = pickle.load(open(filename1, 'rb'))

In [265]:
#accuracy for long Europarl
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_sw.score(x_formal, formal_large.lang)
print(result)

0.9995309806307676


In [266]:
#accuracy for short Europarl
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_sw.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9755861959455883


In [267]:
#accuracy for long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.99550569037589509147
[[29249    86    10    10]
 [   16 32112     3    25]
 [  130   127 30389    34]
 [   27    78    12 31849]]


In [20]:
#accuracy for short TED Talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.98927313480627598885
[[25025   243    19    59]
 [   59 27934    29    61]
 [  180   171 24144    72]
 [   69   157    20 27940]]
